In [ ]:
# default_exp shapley_values

In [ ]:
#export
# Author: Simon Grah <simon.grah@thalesgroup.com>
#         Vincent Thouvenot <vincent.thouvenot@thalesgroup.com>

# MIT License

# Copyright (c) 2020 Thales Six GTS France

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.

In [ ]:
#export
import numpy as np
import pandas as pd
from itertools import combinations
from math import factorial
from tqdm import tqdm

# Shapley Value

> Calculate the exact Shapley Values for an individual $x$ in a game based on a reference $r$ and the reward function $fc$.

## Theory

### Shapley Value definition

In Collaborative Game Theory, Shapley Values ([Shapley,1953]) can distribute a reward among players in a fairly way according to their contribution to the win in a cooperative game. We note $\mathcal{M}$ a set of $d$ players. Moreover, $v : P(\mathcal{M}) \rightarrow R_v$ a reward function such that $v(\emptyset) = 0$. The range $R_v$ can be $\Re$ or a subset of $\Re$. $P(\mathcal{M})$ is a family of sets over $\mathcal{M}$. If $S \subset \mathcal{M}\text{, } v(S)$ is the amount of wealth produced by coalition $S$ when they cooperate.

The Shapley Value of a player $j$ is a fair share of the global wealth $v(\mathcal{M})$ produced by all players together:

$$\phi_j(\mathcal{M},v) = \sum_{S \subset \mathcal{M}\backslash \{j\}}\frac{(d -|S| - 1)!|S|!}{d!}\left(v(S\cup \{j\}) - v(S)\right),$$

with $|S| = \text{cardinal}(S)$, i.e. the number of players in coalition $S$.

### Shapley Values as contrastive local attribute importance in Machine Learning

Let be $X^*\subset\Re^d$ a dataset of individuals where a Machine Learning model $f$ is trained and/or tested and $d$  the dimension of $X^*$. $d>1$ else we do not need to compute Shapley Value. We consider the attribute importance of an individual $\mathbf{x^*} = \{x_1^*, \dots, x_d^*\} \in X^*$ according to a given reference $\mathbf{r} = \{r_1, \dots, r_d\}\in X^*$.  We're looking for $\boldsymbol{\phi}=(\phi_j)_{j\in\{1, \dots, d\}}\in \Re^d$ such that:
$$ \sum_{j=1}^{d} \phi_j = f(\mathbf{x^*}) - f(\mathbf{r}), $$ 
where $\phi_j$ is the attribute contribution of feature indexed $j$.  We loosely identify each feature by its column number. Here the set of players $\mathcal{M}=\{1, \dots, d\}$ is the feature set.

In Machine Learning, a common choice for the reward is $ v(S) = \mathbb{E}[f(X) | X_S = \mathbf{x_S^*}]$, where $\mathbf{x_S^*}=(x_j^*)_{j\in S}$ and $X_S$ the element of $X$ for the coalition $S$. 
For any $S\subset\mathcal{M}$, let's define $ z(\mathbf{x^*},\mathbf{r},S)$ such that $z(\mathbf{x^*},\mathbf{r},\emptyset) = \mathbf{r}$, \ $z(\mathbf{x^*},\mathbf{r},\mathcal{M}) = \mathbf{x^*}$ and

$$ z(\mathbf{x^*},\mathbf{r},S) = (z_1,\dots, z_d) \text{ with } z_i =  x_i^* \text{ if } i \in S \text{ and } r_i  \text{ otherwise }$$ 

As explain in [Merrick,2019], each reference $\textbf{r}$ sets a single-game with $ v(S) = f(z(\mathbf{x^*},\mathbf{r},S)) - f(\mathbf{r}) $, $v(\emptyset) = 0 $ and $v(\mathcal{M}) = f(\mathbf{x^*}) - f(\mathbf{r}) $.

Furthermore, we can extend the previous result by using several references well chosen. In that case, the final Shapley Values obtained are simply the average of those calculated on each reference independantly. But, in order to accelerate the estimation, we modify the algorithm to take into account this situation.  

### References

[Shapley,1953] _A value for n-person games_. Lloyd S Shapley. In Contributions to the Theory of Games, 2.28 (1953), pp. 307 - 317.

[Merrick,2019] _The Explanation Game: Explaining Machine Learning Models with Cooperative Game Theory_. Luke Merrick, Ankur Taly, 2019.

## Function 

__Parameters__

* `x`: pandas Series. The instance $\mathbf{x^*}$ for which we want to calculate Shapley value of each attribute,

* `fc`: python function. The reward function $v$,

* `ref`: pandas Series or pandas DataFrame. Either one or several references $\mathbf{r}$. The Shapley values (attribute importance) is a contrastive explanation according to these individual(s).

__Returns__

* `Φ`: pandas Series. Shapley values of each attribute

In [ ]:
#export
def ShapleyValues(x, fc, ref):
    """
    Calculate the exact Shapley Values for an individual x
    in a game based on a reference r and the reward function fc.
    """

    # Get general information
    feature_names = list(x.index)
    d = len(feature_names) # dimension
    set_features = set(feature_names)

    # Store Shapley Values in a pandas Series
    Φ = pd.Series(np.zeros(d), index=feature_names)

    # Individual reference or dataset of references
    def output_single_ref(coalition, feature_names):
        z = np.array([x[col] if col in coalition else ref.loc[col] for col in feature_names])
        return fc(z)

    def output_several_ref(coalition, feature_names):
        rewards = []
        idxs = np.random.choice(ref.index, size=len(ref), replace=False)
        for idx in idxs:
            z = np.array([x[col] if col in coalition else ref.loc[idx, col] for col in feature_names])
            rewards.append(fc(z))
        return np.mean(rewards)

    if isinstance(ref, pd.core.series.Series):
        individual_ref = True
        output = output_single_ref
    elif isinstance(ref, pd.core.frame.DataFrame):
        if ref.shape[0] == 1:
            ref = ref.iloc[0]
            individual_ref = True
            output = output_single_ref
        else:
            individual_ref = False
            output = output_several_ref

    # Start computation (number of coalitions: 2**d - 1)
    for cardinal_S in tqdm(range(0, d)):
        # weight
        ω = factorial(cardinal_S) * (factorial(d - cardinal_S - 1))
        ω /= factorial(d)
        # iter over all combinations of size cardinal_S
        for S in combinations(feature_names, cardinal_S):
            S = list(S)
            f_S = output(S, feature_names)
            # Consider only features outside of S
            features_out_S = set_features - set(S)
            for j in features_out_S:
                S_union_j = S + [j]
                f_S_union_j = output(S_union_j, feature_names)
                # Update Shapley value of attribute i
                Φ[j] += ω * (f_S_union_j - f_S)

    return Φ

## Example

We use a simulated dataset from the book _Elements of Statistical Learning_ ([hastie,2009], the Radial example). $X_1, \dots , X_{d}$ are standard independent Gaussian. The model is determined by:

$$ Y = \prod_{j=1}^{d} \rho(X_j), $$

where $\rho\text{: } t \rightarrow \sqrt{(0.5 \pi)} \exp(- t^2 /2)$. The regression function $f_{regr}$ is deterministic and simply defined by $f_r\text{: } \textbf{x} \rightarrow \prod_{j=1}^{d} \phi(x_j)$. For a reference $\mathbf{r^*}$ and a target $\mathbf{x^*}$, we define the reward function $v_r^{\mathbf{r^*}, \mathbf{x^*}}$ such as for each coalition $S$, $v_r^{\mathbf{r^*}, \mathbf{x^*}}(S) = f_{regr}(\mathbf{z}(\mathbf{x^*}, \mathbf{r^*}, S)) - f_{regr}(\mathbf{r^*}).$

 [hastie,2009] _The Elements of Statistical Learning: Data Mining, Inference, and Prediction, Second Edition_. Hastie, Trevor and Tibshirani, Robert and Friedman, Jerome. Springer Series in Statistics, 2009.
	

In [ ]:
d, n_samples = 5, 100
mu = np.zeros(d)
Sigma = np.zeros((d,d))
np.fill_diagonal(Sigma, [1] * d)
X = np.random.multivariate_normal(mean=mu, cov=Sigma, size=n_samples)
X = pd.DataFrame(X, columns=['x'+str(i) for i in range(1, d+1)])
def fc(x):
    phi_x = np.sqrt(.5 * np.pi) * np.exp(-0.5 * x ** 2)
    return np.prod(phi_x)
y = np.zeros(len(X))
for i in range(len(X)):
    y[i] = fc(X.values[i])
n = 2**d - 2
print("dimension = {0} ; nb of coalitions = {1}".format(str(d), str(n)))

dimension = 5 ; nb of coalitions = 30


### Pick an individual x to explain

In [ ]:
x = X.iloc[np.random.choice(len(X), size=1)[0],:]
x

x1   -2.090492
x2   -0.762373
x3    0.885185
x4    0.160589
x5   -0.343075
Name: 3, dtype: float64

### Single reference

In [ ]:
reference = X.iloc[np.random.choice(len(X), size=1)[0],:]
reference

x1    1.556253
x2   -0.918537
x3    0.781364
x4   -0.634712
x5   -1.138441
Name: 88, dtype: float64

In [ ]:
true_shap = ShapleyValues(x=x, fc=fc, ref=reference)

100%|██████████| 5/5 [00:00<00:00, 290.51it/s]


In [ ]:
true_shap

x1   -0.190077
x2    0.025982
x3   -0.017088
x4    0.037342
x5    0.117045
dtype: float64

### Several references 

In [ ]:
references = X.iloc[np.random.choice(len(X), size=10, replace=False),:]
references

,x1,x2,x3,x4,x5
96,1.574688,1.494760,0.915205,-0.519904,1.666727
28,-0.420784,1.645722,-0.899554,0.673533,1.375151
55,-0.362972,1.218569,-1.086842,-1.630319,-0.278048
19,0.140422,0.264950,-0.405713,-0.499591,-0.483285
17,-1.225540,1.886731,-0.417747,1.024608,0.727690
85,-1.869827,0.786627,-1.507465,-1.262615,-0.512932
57,-0.589967,-0.279551,-1.773805,0.727048,-0.142259
50,-1.555718,0.407265,-0.105540,0.448243,0.397557
62,0.660258,0.524948,1.599954,-0.878579,1.321833
63,-0.627046,-0.820915,-0.304794,0.828449,0.760209


In [ ]:
true_shaps = ShapleyValues(x=x, fc=fc, ref=references)

100%|██████████| 5/5 [00:00<00:00, 46.64it/s]


In [ ]:
true_shaps

x1   -0.569910
x2    0.031310
x3    0.015586
x4    0.119228
x5    0.080527
dtype: float64

## Tests

In [ ]:
x_pred = fc(x.values)
reference_pred = fc(reference.values)
fcs = []
for r in references.values:
    fcs.append(fc(r))
references_pred = np.mean(fcs)

In [ ]:
assert np.abs(true_shap.sum() - (x_pred - reference_pred)) <= 1e-10 

In [ ]:
assert np.abs(true_shaps.sum() - (x_pred - references_pred)) <= 1e-10

## Export-

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted index.ipynb.
Converted inspector.ipynb.
Converted monte_carlo_shapley.ipynb.
Converted plots.ipynb.
Converted sgd_shapley.ipynb.
Converted shapley_values.ipynb.
